# Dowload e Mineração de Projetos de Lei

1. A *primeira* parte do algorítimo baixa os arquivos em pdf da analisados;
2. A *segunda* estapa busca nos textos desses documentos palavras-chave.

## *Primeira parte: modelo para baixar projetos de lei*

### bibliotecas para a extração

In [2]:
from tqdm.auto import tqdm
import pandas as pd
import requests
import glob 
import os

### importa banco de dados de proposições

Utilizei o link das proposições para baixar os projetos de lei em PDF

In [3]:
arquivo = r'.\pl_evangélicos_carismáticos_Secular_54_55.xlsx'
com = pd.read_excel(arquivo, header=0)

### Algorítimo que faz o dowload dos arquivos pdf.

Vai notar, que cada arquivo possuirá um número, que refere-se ao ID da proposta.
A partir do banco de dados dessas proposições *pl_evangélicos_carismáticos_Secular_54_55.xlsx*, poderá investigar 
outras informações dos PLs.

In [3]:
def dowload_pdf_cd(url, loc=None):
    if loc is None:
            loc = os.path.basename(url.split("?")[1].split("=")[1] +'.pdf') # pega o ID do PL que será salvo com esse número.
            res = requests.get(url, stream=True)
            if res.status_code == requests.codes.OK:
                with open(loc, 'wb') as data:
                    for parte in res.iter_content(chunk_size=256):
                        data.write(parte)
                    else:
                        res.raise_for_status()


if __name__ == "__main__":
    
    for j in tqdm(com['complete_link']): # executa a função anterior
        dowload_pdf_cd(j)


## Segunda parte: data mining

In [37]:
# Insira aqui o diretório onde os arquivos foram baixados

e = r'C:\Users\Andre G\Documents\Pesquisa\Scraping_Deputados\Evangelicos\Analise deputados Evangélicos\Descrição completa da atividade parlamentar evangélicos(15-18)\PLsSecular\Projetos_Analisados'

In [38]:
os.listdir

<function nt.listdir(path=None)>

In [39]:
# lista os endereços com os nomes dos arquivos
# junte o diretório com o nome dos arquivos, para realizar leitura.

endereco = []
for i in os.listdir('./Projetos_Analisados/'): # você poderá incluir o diretório, caso seja necessário.
    local = e + "\\" + i
    endereco.append(local)
endereco[1]

'C:\\Users\\Andre G\\Documents\\Pesquisa\\Scraping_Deputados\\Evangelicos\\Analise deputados Evangélicos\\Descrição completa da atividade parlamentar evangélicos(15-18)\\PLsSecular\\Projetos_Analisados\\1020940.pdf'

In [116]:
# Esse script acessa os endereços gerados anteriormente. 
# Ele abre os documentos, acessa cada página e faz uma busca em seu interior pelo termo.
# O termo de nossa pesauisa foi "empreendedorismo".

#Importa bibliotecas

import os
import PyPDF2
import re

er = [] # lista de arquivos que não foram analisados, por algum erro.

# algorítimo que acessa os pdfs, a partir da lista de endereços gerados anteriomente.
for arquivo in tqdm(endereco[1:]):
    try:
        nome = arquivo.split()[8].split('\\')[3] # copia a numeração referente ao 
        object = PyPDF2.PdfFileReader(arquivo, 'rb')
        
        # obter o número das páginas
        NumPages = object.getNumPages()

        # termos da busca nos pdf´s
        String = "empreendeder|empreendedorismo"

        # extrai o texto e realiza a busca
        for i in range(0, NumPages):
            PageObj = object.getPage(i)
            Text = PageObj.extractText()
            ResSearch = re.search(String, Text)
            if ResSearch == None:
                continue
            else:
                print(["Página: " + str(i) + " | proposta => " + nome, ResSearch])
                                      
    except Exception:
        error = nome
        er.append(error)
        pass
        

['Página: 2 | proposta => 1240635.pdf', <re.Match object; span=(201, 217), match='empreendedorismo'>]
['Página: 2 | proposta => 1318507.pdf', <re.Match object; span=(480, 496), match='empreendedorismo'>]
['Página: 7 | proposta => 900010.pdf', <re.Match object; span=(516, 532), match='empreendedorismo'>]



In [ ]:
# lista de textos não analisados
er